## Classification

In [22]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

import tensorflow as tf
import numpy as np 
from prep_functions import *

np.random.seed(1234)

In [3]:
root_dir = "/home/alberto_sinigaglia/gaia"
mass_range = "CNN_low_mass"

path = f"{root_dir}/{mass_range}_train.npz"

with np.load(path, allow_pickle=False) as data:
    X = data["X"]
    y = data["y"]

In [4]:
X_train, X_val, y_train, y_val = split_train_val(X, y, val_size=0.2)

In [16]:
def make_dataset(X, y, batch_size=32, shuffle=True):
    '''
    This function creates a TensorFlow dataset from numpy arrays.
    '''
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    y = np.reshape(y, (y.shape[0], 1))

    dataset = tf.data.Dataset.from_tensor_slices((X, y))

    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(X))
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return dataset

train_dataset = make_dataset(X_train, y_train, batch_size=64)
val_dataset = make_dataset(X_val, y_val, batch_size=64, shuffle=False)

In [14]:
tf.config.optimizer.set_jit(False)  # call once before building the model

def basic_block(x, kernel_size, filters, name, pool_size=2):
    """Single Conv1D layer with ReLU activation + global avarage pooling"""

    x = tf.keras.layers.Conv1D(filters, kernel_size, padding='same', activation='relu', name=f'{name}_conv')(x)
    x = tf.keras.layers.AveragePooling1D(pool_size,  name=f'{name}_avgpooling')(x)
    return x
    

def residual_block(x, kernel_size, filters, name):
    """Two Conv1D layers + skip connection + ReLU activation"""

    shortcut = x
    x = tf.keras.layers.Conv1D(filters, kernel_size, padding='same', activation='relu', name=f'{name}_conv1')(x)
    x = tf.keras.layers.Conv1D(filters, kernel_size, padding='same', activation=None,  name=f'{name}_conv2')(x)
    x = tf.keras.layers.Add( name=f'{name}_add')([shortcut, x])
    x = tf.keras.layers.Activation('relu', name=f'{name}_relu')(x)
    return x


def build_model(input_length=6144, channels=1, filters=42):

    inp = tf.keras.layers.Input(shape=(input_length, channels), name='input_layer')

    x = basic_block(inp, 16, filters, name='bb_1')
    x = residual_block(x, 16, filters, name='rb_1')

    x = basic_block(x, 32, filters, name='bb_2')
    x = residual_block(x, 32, filters, name='rb_2')

    x = basic_block(x, 64, filters, name='bb_3')
    x =  residual_block(x, 64, filters, name='rb_3')

    x = basic_block(x, 64, filters, name='bb_4')

    x = tf.keras.layers.Flatten(name='flatten_layer')(x)

    x = tf.keras.layers.Dense(128, activation='relu', name='dl_1')(x)
    x = tf.keras.layers.Dense(64,  activation='relu', name='dl_2')(x)
    x = tf.keras.layers.Dense(64,  activation='relu', name='dl_3')(x)
    x = tf.keras.layers.Dense(32,  activation='relu', name='dl_4')(x)

    out = tf.keras.layers.Dense(1, activation='sigmoid', name='output_layer')(x)

    model =  tf.keras.Model(inp, out, name='classification_model')
    return model

model = build_model()
model.summary()

Model: "classification_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 6144, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bb_1_conv (Conv1D)  │ (None, 6144, 42)  │        714 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bb_1_avgpooling     │ (None, 3072, 42)  │          0 │ bb_1_conv[0][0]   │
│ (AveragePooling1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_1_conv1 (Conv1D) │ (None, 3072, 42)  │     28,266 │ bb_1_avgpooling[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_1_conv2 (Conv1D) │ (None, 3072, 42)  │     28,266 │ rb_1_conv1[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_1_add (Add)      │ (None, 3072, 42)  │          0 │ bb_1_avgpooling[… │
│                     │                   │            │ rb_1_conv2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_1_relu           │ (None, 3072, 42)  │          0 │ rb_1_add[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bb_2_conv (Conv1D)  │ (None, 3072, 42)  │     56,490 │ rb_1_relu[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bb_2_avgpooling     │ (None, 1536, 42)  │          0 │ bb_2_conv[0][0]   │
│ (AveragePooling1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_2_conv1 (Conv1D) │ (None, 1536, 42)  │     56,490 │ bb_2_avgpooling[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_2_conv2 (Conv1D) │ (None, 1536, 42)  │     56,490 │ rb_2_conv1[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_2_add (Add)      │ (None, 1536, 42)  │          0 │ bb_2_avgpooling[… │
│                     │                   │            │ rb_2_conv2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_2_relu           │ (None, 1536, 42)  │          0 │ rb_2_add[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bb_3_conv (Conv1D)  │ (None, 1536, 42)  │    112,938 │ rb_2_relu[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bb_3_avgpooling     │ (None, 768, 42)   │          0 │ bb_3_conv[0][0]   │
│ (AveragePooling1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_3_conv1 (Conv1D) │ (None, 768, 42)   │    112,938 │ bb_3_avgpooling[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_3_conv2 (Conv1D) │ (None, 768, 42)   │    112,938 │ rb_3_conv1[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_3_add (Add)      │ (None, 768, 42)   │          0 │ bb_3_avgpooling[… │
│                     │                   │            │ rb_3_conv2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_3_relu           │ (None, 768, 42)   │          0 │ rb_3_add[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bb_4_conv (Conv1D)  │ (None, 768, 42)   │    112,938 │ rb_3_relu[0][0] 

 Total params: 2,757,509 (10.52 MB)

 Trainable params: 2,757,509 (10.52 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.Accuracy(), tf.keras.metrics.TruePositives(), 
                                                                     tf.keras.metrics.FalsePositives(), tf.keras.metrics.Recall()])

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath="best_model.keras", 
        monitor="val_recall",
        mode="max",
        save_best_only=True,
        verbose=0,
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_recall",
        mode="max",
        patience=8,           # stop if no improvement for 8 epochs
        min_delta=1e-4,       # ignore tiny bumps
        restore_best_weights=True,
        verbose=0,
    ),
    tf.keras.callbacks.CSVLogger(
        filename=f"training_log_{mass_range}.csv",
        append=False,
    )
]


In [23]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=200,
    batch_size=128,
    callbacks=callbacks,
    verbose=1,
)

Epoch 1/200
  3/182 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.0000e+00 - false_positives_1: 0.0000e+00 - loss: 0.6933 - recall_1: 0.0000e+00 - true_positives_1: 0.0000e+00

182/182 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.0000e+00 - false_positives_1: 691.0000 - loss: 0.6932 - recall_1: 0.1124 - true_positives_1: 653.0000 - val_accuracy: 0.0000e+00 - val_false_positives_1: 0.0000e+00 - val_loss: 0.6931 - val_recall_1: 0.0000e+00 - val_true_positives_1: 0.0000e+00
Epoch 2/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step - accuracy: 0.0000e+00 - false_positives_1: 2464.0000 - loss: 0.6932 - recall_1: 0.4132 - true_positives_1: 2400.0000 - val_accuracy: 0.0000e+00 - val_false_positives_1: 0.0000e+00 - val_loss: 0.6932 - val_recall_1: 0.0000e+00 - val_true_positives_1: 0.0000e+00
Epoch 3/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step - accuracy: 0.0000e+00 - false_positives_1: 2265.0000 - loss: 0.6932 - recall_1: 0.3814 - true_positives_1: 2215.0000 - val_accuracy: 0.0000e+00 - val_false_positives_1: 0.0000e+00 - val_loss: 0.6931 - val_recall_1: 0.0000e+00 - val_true_positives_1: 0.0000e+00
Epoch 4/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 

In [ ]:
model_path = ''
model = tf.keras.models.load_model(model_path)

y_pred = model.predict(test_X)

In [ ]:
# True Positive Test -- plot noise series

figure_features()
fig = plt.figure(figsize=(10,8))
plt.hist(test_Y_pred[0::2],bins=100,histtype='step',color='red', cumulative=-1, linewidth=2.5, label= "True Positive Test")
plt.hist(test_Y_pred[1::2],bins=100,histtype='step',color='blue', cumulative=-1, linewidth=2.5, label= "False Positive Test")
plt.ylabel('Cumulative Tests', fontsize=25)
plt.xlabel('Threshold', fontsize=25)
plt.yscale("log")
plt.title("Test Metrics on a 200 epoch Low Mass Model", fontsize=25, y=1.0)
plt.grid(linewidth=1, color='black', alpha=0.2)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)



labels = ['True Positive Test', 'False Positive Test']
handle1 = matplotlib.lines.Line2D([], [], c='r')
handle2 = matplotlib.lines.Line2D([], [], c='b')
leg = plt.legend(handles=[handle1, handle2],labels=labels, loc='lower center', prop={'size':20})

# change the line width for the legend
for line in leg.get_lines():
    line.set_linewidth(3.0)

plt.tight_layout()
plt.savefig('Images_Report/Classifier/Low_mass/TP_FP.png', dpi=800, pad_inches=0.1, bbox_inches='tight')
plt.show()
plt.close(fig)

In [ ]:
# Test -- plot noise and signal series

figure_features()
fig = plt.figure(figsize=(10,8)) 
plt.hist(test_Y_pred[1::2],bins=100,histtype='step',color='orange', linewidth=2.5, label= "Noise", hatch='/', facecolor='orange', alpha=0.4, fill=True)
plt.hist(test_Y_pred[0::2],bins=100,histtype='step',color='blue', linewidth=2.5, label= "Signal", hatch='/', facecolor='blue', alpha=0.2, fill=True)
plt.ylabel('Tests', fontsize=25)
plt.xlabel('Score', fontsize=25)
plt.yscale("log")
plt.title("Test on a 200 epoch Low Mass Model", fontsize=25, y=1.0)
plt.grid(linewidth=1, color='black', alpha=0.2)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

labels = ['Noise', 'Signal']
handle1 = matplotlib.lines.Line2D([], [], c='orange')
handle2 = matplotlib.lines.Line2D([], [], c='blue')
leg = plt.legend(handles=[handle1, handle2],labels=labels, loc='upper center', prop={'size':20})

# change the line width for the legend
for line in leg.get_lines():
    line.set_linewidth(3.0)


plt.tight_layout()
plt.savefig('Images_Report/Classifier/Low_mass/Test_general.png', dpi=800, pad_inches=0.1, bbox_inches='tight')
plt.show()
plt.close(fig)